In [ ]:
import pandas as pd
import numpy as np

%run Util.ipynb
%run Metric.ipynb

class BreakSectionManager:
    
    def __init__(self, symbol, level, short, mid):
        self.symbol = symbol
        self.level = level
        self.short = short
        self.mid = mid
        
        self.metric = Metric(symbol,level,short,mid)
        
        self.start_date = self.metric.get_stock_start_date()
        self.end_date = self.metric.get_stock_end_date()
        
        self.section_df = pd.DataFrame(columns=break_section_columns)

        
        
    def analyse_break_for_one_specific_day(self,cur_date):
        broken = self.metric.does_break_highest(cur_date)
        if(broken==True):
            end_date = self.metric.next_break_lowest_date(cur_date)
            if(end_date is not None):
                part = self.metric.get_k_series_between(cur_date,end_date)
                max_price_date = self.metric.get_max_price_date(cur_date,end_date)
                min_price_date = self.metric.get_min_price_date(cur_date,end_date)
                
                atr_value = self.metric.atr(cur_date)
                
                start_price = self.metric.get_cur_price(cur_date)
                highest_price = max_price_date['high']
                lowest_price = min_price_date['low']
                
                mfe = (highest_price - start_price)/atr_value
                mae = (start_price - lowest_price)/atr_value
            
                break_section = [self.symbol,cur_date,end_date,self.metric.get_cur_price(cur_date),
                                self.metric.get_cur_price(end_date),
                                 max_price_date['date'],max_price_date['high'],
                                 min_price_date['date'],min_price_date['low'],
                                mfe,mae]

                return break_section
        
    def build_file_path(self):
        return  break_sections_dir + self.level + '/' + self.level+'_'+str(self.short)+'_'+str(self.mid)+'_' 'sections_'+break_section_version+'_'+ self.symbol +'.txt'
    
    
    def get_all_break_sections(self):
        file_path = self.build_file_path()
        
        if(os.path.exists(file_path)):
            self.section_df = pd.read_csv(file_path)
            
            self.section_df['d_s_datetime'] = pd.to_datetime(self.section_df['d_s_datetime'])
            self.section_df['d_e_datetime'] = pd.to_datetime(self.section_df['d_e_datetime'])
        else:
            print('build '+self.symbol+' break sections')
            self.build_all_break_sections()
                    
            file_path = self.build_file_path()
            self.section_df.to_csv(file_path, index=0)

        return self.section_df
        
        
    def build_all_break_sections(self):
        print('start to analyse '+self.level+' level '+ self.symbol+' all break sections...')
        all_dates = self.metric.list_stock_all_dates()[self.mid:]
        all_dates = set(all_dates)
       
        start_datetime = to_datetime(self.start_date)
        end_datetime = to_datetime(self.end_date)

        while(True):
            cur_date = start_datetime.strftime(YMD_format)
            if(cur_date in all_dates):
                section = self.analyse_break_for_one_specific_day(cur_date)
                if(section is not None):
                    end_date = section[2]
                    
                    if(end_date is not None):
                        self.section_df.loc[self.section_df.shape[0] +1] = section
                        start_datetime = to_datetime(end_date)
                    else:
                        break
                else:
                    start_datetime += dt.timedelta(days = 1)

                if(start_datetime>end_datetime):
                    break
            else:
                start_datetime += dt.timedelta(days = 1)

        self.section_df['d_s_datetime'] = pd.to_datetime(self.section_df['d_s_date'])
        self.section_df['d_e_datetime'] = pd.to_datetime(self.section_df['d_e_date'])

        